In [20]:
import pandas as pd

In [21]:
michelin_restaurants_path = 'data/cleaned/michelin_cleaned.csv'
yelp_restaurants_path = 'data/cleaned/yelp_cleaned.csv'
yelp_reviews_path = 'data/cleaned/yelp_reviews_cleaned.csv'
yelp_users_path = 'data/cleaned/yelp_users_cleaned.csv'

In [22]:
michelin_df = pd.read_csv(michelin_restaurants_path)
yelp_df = pd.read_csv(yelp_restaurants_path)
yelp_reviews_df = pd.read_csv(yelp_reviews_path)
yelp_users_df = pd.read_csv(yelp_users_path)

/var/folders/6c/8xxnyjp91wv66mwmrf9k70nc0000gn/T/ipykernel_5022/616360392.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  yelp_users_df = pd.read_csv(yelp_users_path)


In [23]:
# Extract Michelin LocationInfo (unique locations)
michelin_location_info = michelin_df[['Address', 'Latitude', 'Longitude', 'Location']].drop_duplicates()
michelin_location_info.columns = ['Address', 'Latitude', 'Longitude', 'Location']
print(f"Michelin LocationInfo rows: {len(michelin_location_info):,}")
michelin_location_info.head()

Michelin LocationInfo rows: 18,614


,Address,Latitude,Longitude,Location
0,"Mietenkamer Straße 65, Grassau, 83224, Germany",47.785630,12.465618,"Grassau, Germany"
1,"Burgstraße 5, Munich, 80331, Germany",48.137597,11.577475,"Munich, Germany"
2,"Tonbachstraße 237, Baiersbronn, 72270, Germany",48.536911,8.358280,"Baiersbronn, Germany"
3,"Schlossstraße 27, Perl, 66706, Germany",49.535173,6.387211,"Perl, Germany"
4,"Parkstraße 1, Wolfsburg, 38440, Germany",52.432849,10.789979,"Wolfsburg, Germany"


In [24]:
# Extract Michelin RestaurantInfo
michelin_restaurant_info = michelin_df[[
    'Name', 'Address', 'FacilitiesAndServices', 'Price', 'Cuisine', 
    'PhoneNumber', 'Url', 'WebsiteUrl', 'Award', 'GreenStar', 'Description'
]].copy()
print(f"Michelin RestaurantInfo rows: {len(michelin_restaurant_info):,}")
michelin_restaurant_info.head()

Michelin RestaurantInfo rows: 18,715


,Name,Address,FacilitiesAndServices,Price,Cuisine,PhoneNumber,Url,WebsiteUrl,Award,GreenStar,Description
0,ES:SENZ,"Mietenkamer Straße 65, Grassau, 83224, Germany","Air conditioning,Car park,Interesting wine list",€€€€,"Creative, Modern Cuisine",4.986414e+11,https://guide.michelin.com/en/bayern/grassau/r...,https://www.das-achental.com/,3 Stars,0,"From the get-go, the stage is set in grand sty..."
1,Tohru in der Schreiberei,"Burgstraße 5, Munich, 80331, Germany","Interesting wine list,Notable sake list",€€€€,"Modern Cuisine, Japanese Contemporary",4.989215e+11,https://guide.michelin.com/en/bayern/mnchen/re...,https://schreiberei-muc.de/,3 Stars,0,It is absolutely worth climbing the 23 steps o...
2,Schwarzwaldstube,"Tonbachstraße 237, Baiersbronn, 72270, Germany","Air conditioning,Car park,Great view,Interesti...",€€€€,"Classic French, Creative",4.974425e+11,https://guide.michelin.com/en/baden-wurttember...,https://www.traube-tonbach.de/restaurants-bar/...,3 Stars,0,"Schwarzwaldstube, the flagship restaurant of t..."
3,Victor's Fine Dining by christian bau,"Schlossstraße 27, Perl, 66706, Germany","Air conditioning,Car park,Interesting wine lis...",€€€€,"Creative, Contemporary",4.968668e+10,https://guide.michelin.com/en/saarland/perl/re...,https://www.victors-fine-dining.de/,3 Stars,0,The way in which Christian Bau combines influe...
4,Aqua,"Parkstraße 1, Wolfsburg, 38440, Germany","Air conditioning,Car park,Interesting wine lis...",€€€€,"Creative, Modern Cuisine",4.953616e+11,https://guide.michelin.com/en/niedersachsen/wo...,https://www.restaurant-aqua.com,3 Stars,0,The Ritz-Carlton is located in the heart of Vo...


In [25]:
# Extract Yelp LocationInfo (unique locations)
yelp_location_info = yelp_df[[
    'address', 'city', 'state', 'postal_code', 'latitude', 'longitude'
]].drop_duplicates()
print(f"Yelp LocationInfo rows: {len(yelp_location_info):,}")
yelp_location_info.head()

Yelp LocationInfo rows: 143,554


,address,city,state,postal_code,latitude,longitude
0,"1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197
1,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695
2,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452
3,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564
4,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659


In [26]:
# Extract Yelp RestaurantInfo
yelp_restaurant_info = yelp_df[[
    'business_id', 'name', 'address', 'city', 'state', 'postal_code',
    'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours'
]].copy()

# Convert 'attributes' and 'hours' columns to valid JSON (double quotes)
import ast, json
for col in ['attributes', 'hours']:
    if col in yelp_restaurant_info.columns:
        def fix_json(val):
            if pd.isnull(val):
                return None
            try:
                d = ast.literal_eval(val) if isinstance(val, str) else val
                return json.dumps(d, ensure_ascii=False)
            except Exception:
                return val
        yelp_restaurant_info[col] = yelp_restaurant_info[col].apply(fix_json)

print(f"Yelp RestaurantInfo rows: {len(yelp_restaurant_info):,}")
yelp_restaurant_info.head()

Yelp RestaurantInfo rows: 150,346


,business_id,name,address,city,state,postal_code,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,5.0,7,0,"{""ByAppointmentOnly"": ""True""}","Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,3.0,15,1,"{""BusinessAcceptsCreditCards"": ""True""}","Shipping Centers, Local Services, Notaries, Ma...","{""Monday"": ""0:0-0:0"", ""Tuesday"": ""8:0-18:30"", ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,3.5,22,0,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{""Monday"": ""8:0-22:0"", ""Tuesday"": ""8:0-22:0"", ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,4.0,80,1,"{""RestaurantsDelivery"": ""False"", ""OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{""Monday"": ""7:0-20:0"", ""Tuesday"": ""7:0-20:0"", ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,4.5,13,1,"{""BusinessAcceptsCreditCards"": ""True"", ""Wheelc...","Brewpubs, Breweries, Food","{""Wednesday"": ""14:0-22:0"", ""Thursday"": ""16:0-2..."


In [27]:
# Format Yelp UserInfo (keep only needed columns, drop elite if still present)
yelp_user_info = yelp_users_df[[
    'user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny', 'cool',
    'fans', 'compliment_more', 'compliment_profile', 'compliment_cute', 
    'compliment_list', 'compliment_note', 'compliment_plain', 'compliment_cool',
    'compliment_funny', 'compliment_writer', 'compliment_photos'
]].copy()

# Handle elite and friends as TEXT (comma-separated or JSON string)
if 'elite' in yelp_users_df.columns:
    yelp_user_info['elite'] = yelp_users_df['elite'].astype(str)
else:
    yelp_user_info['elite'] = None

print(f"Yelp UserInfo rows: {len(yelp_user_info):,}")
yelp_user_info.head()

Yelp UserInfo rows: 557,536


,user_id,name,review_count,yelping_since,useful,funny,cool,fans,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos,elite
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,267,65,55,56,18,232,844,467,467,239,180,2007
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,3138,264,184,157,251,1847,7054,3131,3131,1521,1946,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2..."
2,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,28,4,1,6,2,12,16,26,26,10,9,"2009,2010,2011"
3,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,1,1,0,0,0,1,1,0,0,0,0,nan
4,q_QQ5kBBwlCcbL1s4NVK3g,Jane,1221,2005-03-14 20:26:35,14953,9940,11211,1357,163,191,361,147,1212,5696,2543,2543,815,323,"2006,2007,2008,2009,2010,2011,2012,2013,2014"


In [28]:
# Format Yelp Reviews
yelp_reviews_info = yelp_reviews_df[[
    'review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny', 'cool', 'text', 'date'
]].copy()
print(f"Yelp Reviews rows: {len(yelp_reviews_info):,}")
yelp_reviews_info.head()

Yelp Reviews rows: 998,611


,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,J5Q1gH4ACCj6CtQG7Yom7g,56gL9KEJNHiSDUoyjk2o3Q,8yR12PNSMo6FBYx1u5KPlw,2,1,0,0,Went for lunch and found that my burger was me...,2018-04-04 21:09:53
1,HlXP79ecTquSVXmjM10QxQ,bAt9OUFX9ZRgGLCXG22UmA,pBNucviUkNsiqhJv5IFpjg,5,0,0,0,I needed a new tires for my wife's car. They h...,2020-05-24 12:22:14
2,JBBULrjyGx6vHto2osk_CQ,NRHPcLq2vGWqgqwVugSgnQ,8sf9kv6O4GgEb0j1o22N1g,5,0,0,0,Jim Woltman who works at Goleta Honda is 5 sta...,2019-02-14 03:47:48
3,U9-43s8YUl6GWBFCpxUGEw,PAxc0qpqt5c2kA0rjDFFAg,XwepyB7KjJ-XGJf0vKc6Vg,4,0,0,0,Been here a few times to get some shrimp. The...,2013-04-27 01:55:49
4,8T8EGa_4Cj12M6w8vRgUsQ,BqPR1Dp5Rb_QYs9_fz9RiA,prm5wvpp0OHJBlrvTj9uOg,5,0,0,0,This is one fantastic place to eat whether you...,2019-05-15 18:29:25


In [29]:
# Export all normalized tables to CSV for PostgreSQL
output_dir = 'data/postgres'
import os
os.makedirs(output_dir, exist_ok=True)

# Export Michelin tables
michelin_location_path = os.path.join(output_dir, 'michelin_location_info.csv')
michelin_location_info.to_csv(michelin_location_path, index=False)
print(f"✓ Michelin LocationInfo → {michelin_location_path}")

michelin_restaurant_path = os.path.join(output_dir, 'michelin_restaurant_info.csv')
michelin_restaurant_info.to_csv(michelin_restaurant_path, index=False)
print(f"✓ Michelin RestaurantInfo → {michelin_restaurant_path}")

# Export Yelp tables
yelp_location_path = os.path.join(output_dir, 'yelp_location_info.csv')
yelp_location_info.to_csv(yelp_location_path, index=False)
print(f"✓ Yelp LocationInfo → {yelp_location_path}")

yelp_restaurant_path = os.path.join(output_dir, 'yelp_restaurant_info.csv')
yelp_restaurant_info.to_csv(yelp_restaurant_path, index=False)
print(f"✓ Yelp RestaurantInfo → {yelp_restaurant_path}")

yelp_user_path = os.path.join(output_dir, 'yelp_user_info.csv')
yelp_user_info.to_csv(yelp_user_path, index=False)
print(f"✓ Yelp UserInfo → {yelp_user_path}")

yelp_reviews_path_out = os.path.join(output_dir, 'yelp_review_info.csv')
yelp_reviews_info.to_csv(yelp_reviews_path_out, index=False)
print(f"✓ Yelp Reviews → {yelp_reviews_path_out}")

print("\n=== EXPORT SUMMARY ===")
print(f"Michelin LocationInfo: {len(michelin_location_info):,} rows")
print(f"Michelin RestaurantInfo: {len(michelin_restaurant_info):,} rows")
print(f"Yelp LocationInfo: {len(yelp_location_info):,} rows")
print(f"Yelp RestaurantInfo: {len(yelp_restaurant_info):,} rows")
print(f"Yelp UserInfo: {len(yelp_user_info):,} rows")
print(f"Yelp ReviewInfo: {len(yelp_reviews_info):,} rows")

✓ Michelin LocationInfo → data/postgres/michelin_location_info.csv
✓ Michelin RestaurantInfo → data/postgres/michelin_restaurant_info.csv
✓ Yelp LocationInfo → data/postgres/yelp_location_info.csv
✓ Yelp RestaurantInfo → data/postgres/yelp_restaurant_info.csv
✓ Yelp UserInfo → data/postgres/yelp_user_info.csv
✓ Yelp Reviews → data/postgres/yelp_review_info.csv

=== EXPORT SUMMARY ===
Michelin LocationInfo: 18,614 rows
Michelin RestaurantInfo: 18,715 rows
Yelp LocationInfo: 143,554 rows
Yelp RestaurantInfo: 150,346 rows
Yelp UserInfo: 557,536 rows
Yelp ReviewInfo: 998,611 rows
